In [16]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3 as lite
import os

In [2]:
# First page to crawl
url = 'https://dogvacay.com/dog-boarding--tx--austin'

In [17]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'}

In [18]:
# TODO: set up a loop over other cities too

In [19]:
cities = {'tx' : "austin"
          , 'ca' : 'los-angeles'
          , 'ga' : 'atlanta'
          , 'ma' : 'boston'
          , 'nc' : 'charlotte'
          , 'tx' : 'dallas'
          , 'co' : 'denver'
          , 'tx' : 'houston'
          , 'fl' : 'miami'
          , 'mn' : 'minneapolis'
          , 'ny' : 'new-york'
          , 'pa' : 'philadelphia'
          , 'az' : 'phoenix'
          , 'or' : 'portland'
          , 'ca' : 'san-francisco'
          , 'ca' : 'san-diego'
          , 'wa' : 'seattle'
          , 'dc' : 'washington'
         }

In [36]:
page = 1
for k, v in cities.iteritems():
    print k, v
    page = 1
    running = True
    while running:
        url = "https://dogvacay.com/dog-boarding--" + k + "--" + v + "?p="+str(page)
        #print url
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text)
        time.sleep(1)
        pagination_links = soup.findAll('a', {'class': 'pagination-link'})
        running = False
        for pl in pagination_links:
            if pl.text.find('Next') == 0:
                running = True
        page+=1

mn minneapolis
ma boston
wa seattle
tx houston
ca san-diego
nc charlotte
dc washington
ny new-york
pa philadelphia


KeyboardInterrupt: 

In [20]:
# This gets the page from the web, and also waits one second after you do
# The one second wait is to make sure you don't accidentally make too many requests which is bad netiquette
r = requests.get(url, headers=headers)
time.sleep(1)

In [21]:
# TODO: set up a loop until you've checked all pages

In [21]:
# Convert the text to a beautiful soup object
soup = BeautifulSoup(r.content)

In [24]:
# Remeber the "inspect element" trick I showed you in Chrome?  Use that to find the data you want
sitters = soup.findAll('div', {'class': 'card-content'})
len(sitters)

15

In [30]:
# Make sure you got a 200 response, otherwise it indicates an error
r.status_code

200

In [23]:
# Data extraction phase
# TODO: This is a work in progress.  You should have a go at getting it working.
# TODO: Pick which important fields you want to extract, find them, build the data frame
page = 1
for k, v in cities.iteritems():
    print k, v
    page = 1
    running = True
    while running:
        url = "https://dogvacay.com/dog-boarding--" + k + "--" + v + "?p="+str(page)
        #print url
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text)
        time.sleep(1)
        pagination_links = soup.findAll('a', {'class': 'pagination-link'})
        running = False
        for pl in pagination_links:
            if pl.text.find('Next') == 0:
                running = True
        page+=1
        sitter = sitters[3]
        times = []
        fees = []
        reviews = []
        repeats = []
        city = []
        service = []
        rating = []

        for sitter in sitters:
            response_time = sitter.findAll('div', {'class': 'icon-wrapper'})[0].text.strip('\n')[19:]
            fee = sitter.findAll('div', {'class': 'price price--primary u-text-center'})
            review = sitter.find('span', {'class': 'vcard-review'})
            if review is not None and len(review) > 0:
                review = int(review.text.replace('Reviews', '').replace('Review', '').strip())
            else:
                review = 0
            repeat = sitter.findAll('div', {'class': 'icon-wrapper'})
            if len(repeat) == 2:
                repeat = repeat[1].text.strip('\n')[1]
            else:
                repeat = 0
            services = sitter.findAll('div', {'class': 'vcard-line-item'})
            full_star = sitter.findAll('i', {'class': 'ck-star'})
            ratings = len(full_star)
            half_star = sitter.findAll('i', {'class': 'ck-star-half'})
            if len(half_star) > 0:
                ratings += .5
            fees.append(fee[0].text.strip('\n$')[:2])
            city.append('Austin')
            times.append(response_time)
            reviews.append(review)
            repeats.append(repeat)
            service.append(services[-1].text.strip('\n'))
            rating.append(ratings)


        df = pd.DataFrame({'city': city, 'fee': fees, 'response_time': times, 'review' : reviews, 'repeat' : repeats, 'services' : service, 'ratings': rating})
df

,city,fee,ratings,repeat,response_time,review,services
0,Austin,40,5,3,Day,68,"Boarding, Daycare, and Pick up and Drop off"
1,Austin,35,0,0,Few Hours,0,"Boarding, Daycare, and Bathing"
2,Austin,40,5,0,Few Hours,1,Boarding and Daycare
3,Austin,40,5,0,Day,3,"Boarding, Daycare, Checkups and Visits, Walkin..."
4,Austin,35,0,0,Few Hours,0,Boarding and Walking
5,Austin,35,5,0,Few Hours,1,"Boarding, Daycare, Checkups and Visits, Walkin..."
6,Austin,40,5,2,Few Hours,34,"Boarding, Sitting, Daycare, Checkups and Visit..."
7,Austin,55,5,1,Day,21,"Boarding, Training, and Pick up and Drop off"
8,Austin,40,5,1,Day,16,Boarding
9,Austin,30,5,5,Few Hours,8,"Boarding, Daycare, Bathing, and Pick up and Dr..."


In [ ]:
#for k,v in cities.iteritems():
    running = True
    while running:
        url = "https://dogvacay.com/dog-boarding--" + k + "--" + v + "?p="+str(page)
        print url
        filename = k + '-' + v + '.htm'
        if os.path.isfile(filename):
            f = open(filename, 'r')
            data = f.read()
            f.close()
        else:
            r = requests.get(url, headers=headers)
            time.sleep(1)
            f = open(filename, 'w')
            f.write(r.text)
            f.close()
            data = r.text
        soup = BeautifulSoup(data)
        pagination_links = soup.findAll('a', {'class': 'pagination-link'})
        running = False
        for pl in pagination_links:
            if pl.text.find('Next') == 0:
                running = True
        page+=1


In [ ]:
#df.to_csv('dog-vacay.csv', sep='\t')
#save to csv file